In [1]:
import requests
from bs4 import BeautifulSoup
import reverse_geocode
import folium
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib as mpl
import pandas as pd
from sklearn import datasets
import geocoder

In [2]:
Raleigh_Neighborhoods=[]
response_obj=requests.get('https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Raleigh,_North_Carolina').text
soup=BeautifulSoup(response_obj,'lxml')
mydivs = soup.findAll("div", {"class": "mw-category-group"})
for div in mydivs:
    for l in div.findAll('a'):
        for title in l:
            Raleigh_Neighborhoods.append(title)
Raleigh_Neighborhoods.remove('Raleigh, North Carolina neighborhoods')
Raleigh_Neighborhoods=[s.replace('Historic District','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.replace('(North Carolina)','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.replace('(Raleigh, North Carolina)','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.strip(' ') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s+(',Raleigh, North Carolina') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods

['Battery Heights,Raleigh, North Carolina',
 'Bloomsbury,Raleigh, North Carolina',
 'Boylan Heights,Raleigh, North Carolina',
 'Cameron Park,Raleigh, North Carolina',
 'Cameron Village,Raleigh, North Carolina',
 'Cameron Village,Raleigh, North Carolina',
 'Capitol Area,Raleigh, North Carolina',
 'Capitol Heights,Raleigh, North Carolina',
 'Centennial Campus of North Carolina State University,Raleigh, North Carolina',
 'Depot,Raleigh, North Carolina',
 'East Raleigh–South Park,Raleigh, North Carolina',
 'Fayetteville Street (Raleigh),Raleigh, North Carolina',
 'Fayetteville Street,Raleigh, North Carolina',
 'Five Points Historic Neighborhoods,Raleigh, North Carolina',
 'Glenwood Avenue,Raleigh, North Carolina',
 'Glenwood–Brooklyn,Raleigh, North Carolina',
 'Hayes Barton,Raleigh, North Carolina',
 'Hi-Mount,Raleigh, North Carolina',
 'Historic Oakwood,Raleigh, North Carolina',
 'Longview Gardens,Raleigh, North Carolina',
 'Madonna Acres,Raleigh, North Carolina',
 'Maiden Lane,Raleigh, N

In [3]:
df_ral=pd.DataFrame(Raleigh_Neighborhoods)
df_ral=df_ral.rename(columns={0:'Neighborhoods'})
df_ral=df_ral.drop_duplicates(keep='first')
df_ral

,Neighborhoods
0,"Battery Heights,Raleigh, North Carolina"
1,"Bloomsbury,Raleigh, North Carolina"
2,"Boylan Heights,Raleigh, North Carolina"
3,"Cameron Park,Raleigh, North Carolina"
4,"Cameron Village,Raleigh, North Carolina"
6,"Capitol Area,Raleigh, North Carolina"
7,"Capitol Heights,Raleigh, North Carolina"
8,Centennial Campus of North Carolina State Univ...
9,"Depot,Raleigh, North Carolina"
10,"East Raleigh–South Park,Raleigh, North Carolina"


In [4]:
geolocator=Nominatim(user_agent='Raleigh')
city=[]
lat=[]
long=[]
from geopy.exc import GeocoderTimedOut
try:
    for i in Raleigh_Neighborhoods:
        location = geolocator.geocode(i)
        if location is not None:
            city.append(i)
            lat.append(location.latitude)
            long.append(location.longitude)
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(location, e.msg))

In [5]:
df=pd.DataFrame({'Neighborhoods':city,'latitude':lat,'longitude':long})

In [6]:
df=df.drop_duplicates(subset='latitude',keep='first')

In [7]:
df

,Neighborhoods,latitude,longitude
0,"Battery Heights,Raleigh, North Carolina",35.777058,-78.617563
1,"Bloomsbury,Raleigh, North Carolina",35.808897,-78.648599
2,"Boylan Heights,Raleigh, North Carolina",35.774159,-78.652102
3,"Cameron Park,Raleigh, North Carolina",35.785779,-78.655470
4,"Cameron Village,Raleigh, North Carolina",35.790361,-78.660413
6,"Capitol Area,Raleigh, North Carolina",35.789884,-78.621851
8,"Depot,Raleigh, North Carolina",35.780398,-78.639099
9,"Fayetteville Street (Raleigh),Raleigh, North C...",35.768875,-78.641336
11,"Glenwood Avenue,Raleigh, North Carolina",35.886724,-78.744499
12,"Hayes Barton,Raleigh, North Carolina",35.802755,-78.646583


In [14]:
raleigh=['Northwest Raleigh,Raleigh,nc','North Raleigh,Raleigh,nc','Northeast raleigh,Raleigh,nc','Southwest raleigh,Raleigh,nc','Southeast raleigh,Raleigh,nc'
        ,'east raleigh,Raleigh,nc']

In [15]:
raliegh_side=[]
lat=[]
long=[]
from geopy.exc import GeocoderTimedOut
try:
    for i in raleigh:
        location = geolocator.geocode(i)
        if location is not None:
            raliegh_side.append(i)
            lat.append(location.latitude)
            long.append(location.longitude)
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(location, e.msg))

In [78]:
df_ral=pd.DataFrame({'Raleigh_Side':raliegh_side,'latitude':lat,'longitude':long})
df_ral

,Raleigh_Side,latitude,longitude
0,"Northwest Raleigh,Raleigh,nc",37.756782,-78.600658
1,"North Raleigh,Raleigh,nc",37.756782,-78.600658
2,"Northeast raleigh,Raleigh,nc",37.756782,-81.174266
3,"Southwest raleigh,Raleigh,nc",37.756782,-78.702732
4,"Southeast raleigh,Raleigh,nc",37.756782,-78.596097
5,"east raleigh,Raleigh,nc",37.756782,-81.174266


In [86]:
CLIENT_ID = 'EA0WZQZP3HJVT4DTV02NLEGNIFSAHPTRXK50TAZIIYY5A5OG' # your Foursquare ID
CLIENT_SECRET = '5X1OY3KPXNEM1F5CVU2A124DQUNH3GC0TDP1MP1M0FX31D1Z' # your Foursquare Secret

# CLIENT_ID ='BCZ1DKRCRZTMRZPSY5HHEQTQJNZCITUWFO3CLRCIUQOOGSFB'
# CLIENT_SECRET ='WPWOCZ3QZVXYQCXPKBPKSKW1CQSFBLTKAWYGSS0WBXJKZZZW'
VERSION = '20190104'
LIMIT = 200
radius= 5000
latitude = 35.787743
longitude = -78.644257
search_query='Restaurants'
restID=[]
restName=[]
restCity=[]
restLat=[]
restLng=[]
restRate=[]

In [87]:
url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude,longitude,VERSION,search_query,radius,LIMIT)
results = requests.get(url1).json()
r=results['response']['groups'][0]['items']
for a in range(len(r)):
     print(r[a]['venue']['name'],r[a]['venue']['name'])

Mellow Mushroom Mellow Mushroom
Jersey Mike's Subs Jersey Mike's Subs
MoJoe's Burger Joint MoJoe's Burger Joint
Sullivan's Steakhouse Sullivan's Steakhouse
Bad Daddy's Bad Daddy's
Vidrio Vidrio
My Way Tavern My Way Tavern
42nd St Oyster Bar 42nd St Oyster Bar
Layered Croissanterie Layered Croissanterie
Krispy Kreme Krispy Kreme
Yellow Dog Bread Company Yellow Dog Bread Company
Crawford and Son Crawford and Son
J. Betski's J. Betski's
Taverna Agora Taverna Agora
Char-Grill Char-Grill
Stanbury Stanbury
Second Empire Restaurant & Tavern Second Empire Restaurant & Tavern
Cortez Seafood & Cocktail Bar Cortez Seafood & Cocktail Bar
Centro Centro
Bida Manda Laotian Restaurant and Bar Bida Manda Laotian Restaurant and Bar
Death & Taxes Death & Taxes
La Santa Modern Mexican Food La Santa Modern Mexican Food
lucettegrace lucettegrace
Beasley's Chicken + Honey Beasley's Chicken + Honey
Sitti Sitti
Cousins Maine Lobster Cousins Maine Lobster
Tazza Kitchen Cameron Village Tazza Kitchen Cameron Vill

In [73]:
# for lat ,lng in zip(df_ral['latitude'],df_ral['longitude']):  
#     url1 = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, lat,lng, VERSION)
#     results = requests.get(url1).json()
#     print(results)

{'meta': {'code': 200, 'requestId': '5e275a0014a126001b6cff60'}, 'response': {'venues': []}}
{'meta': {'code': 200, 'requestId': '5e2759b1882fc7001bd25e09'}, 'response': {'venues': []}}
{'meta': {'code': 200, 'requestId': '5e2759b6e826ac001b655d9a'}, 'response': {'venues': []}}
{'meta': {'code': 200, 'requestId': '5e2759d43907e70022594706'}, 'response': {'venues': []}}
{'meta': {'code': 200, 'requestId': '5e275a14216785001b02a5e2'}, 'response': {'venues': []}}
{'meta': {'code': 200, 'requestId': '5e2759b1882fc7001bd26125'}, 'response': {'venues': []}}


In [104]:
for lat ,lng in zip(df['latitude'],df['longitude']):  
    url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat,lng,VERSION,search_query,radius,LIMIT)
    results = requests.get(url1).json()
    r=results['response']['groups'][0]['items']
    for a in range(len(r)):
        print(r[a]['venue'])
         #print(r[a]['venue']['name'],r[a]['venue']['categories'][0]['name'],r[a]['venue'])
         

{'id': '4b42a6a9f964a52044d825e3', 'name': 'Cook Out', 'location': {'address': '1201 New Bern Ave', 'crossStreet': 'at Tarboro St.', 'lat': 35.780185086172985, 'lng': -78.6217955650056, 'labeledLatLngs': [{'label': 'display', 'lat': 35.780185086172985, 'lng': -78.6217955650056}], 'distance': 517, 'postalCode': '27610', 'cc': 'US', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['1201 New Bern Ave (at Tarboro St.)', 'Raleigh, NC 27610', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d16e941735', 'name': 'Fast Food Restaurant', 'pluralName': 'Fast Food Restaurants', 'shortName': 'Fast Food', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_', 'suffix': '.png'}, 'primary': True}], 'delivery': {'id': '1474037', 'url': 'https://www.grubhub.com/restaurant/cook-out-1201-new-bern-avenue-raleigh/1474037?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=1474037', 'provider'

{'id': '4ad91c4ff964a520311821e3', 'name': 'NOFO @ the Pig', 'location': {'address': '2014 Fairview Rd', 'crossStreet': 'at Five Points', 'lat': 35.80508061463781, 'lng': -78.6468957289808, 'labeledLatLngs': [{'label': 'display', 'lat': 35.80508061463781, 'lng': -78.6468957289808}], 'distance': 451, 'postalCode': '27608', 'cc': 'US', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['2014 Fairview Rd (at Five Points)', 'Raleigh, NC 27608', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d16d941735', 'name': 'Café', 'pluralName': 'Cafés', 'shortName': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}, 'primary': True}], 'delivery': {'id': '1312861', 'url': 'https://www.grubhub.com/restaurant/nofo--the-pig-2014-fairview-road-raleigh/1312861?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=1312861', 'provider': {'name': 'grubhub', 'icon': {'pref

{'id': '532ac29b498ec61be94e2292', 'name': 'Boulted Bread', 'location': {'address': '614 W South St', 'lat': 35.77253500609877, 'lng': -78.64872944350167, 'labeledLatLngs': [{'label': 'display', 'lat': 35.77253500609877, 'lng': -78.64872944350167}], 'distance': 354, 'postalCode': '27603', 'cc': 'US', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['614 W South St', 'Raleigh, NC 27603', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d16a941735', 'name': 'Bakery', 'pluralName': 'Bakeries', 'shortName': 'Bakery', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}
{'id': '52251fd611d21c8d037f3127', 'name': 'José and Sons', 'location': {'address': '327 - 102 W Davie St', 'lat': 35.77541026824044, 'lng': -78.64478241052932, 'labeledLatLngs': [{'label': 'display', 'lat': 35.77541026824044, 'lng': -78.64478241052932}], 'distance': 675, 'postalCode': '27

{'id': '561e6366498e6af1a3ef6396', 'name': 'My Way Tavern', 'location': {'address': '522 Saint Marys St', 'lat': 35.78727965220433, 'lng': -78.65067134711106, 'labeledLatLngs': [{'label': 'display', 'lat': 35.78727965220433, 'lng': -78.65067134711106}], 'distance': 464, 'postalCode': '27605', 'cc': 'US', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['522 Saint Marys St', 'Raleigh, NC 27605', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d14e941735', 'name': 'American Restaurant', 'pluralName': 'American Restaurants', 'shortName': 'American', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}
{'id': '528a6b0e11d27a62a2728622', 'name': 'Tazza Kitchen Cameron Village', 'location': {'address': '432 Woodburn Rd', 'crossStreet': 'Cameron Street', 'lat': 35.7904985, 'lng': -78.65862159999999, 'labeledLatLngs': [{'label': 'display', 'lat': 35.790498

{'id': '528a6b0e11d27a62a2728622', 'name': 'Tazza Kitchen Cameron Village', 'location': {'address': '432 Woodburn Rd', 'crossStreet': 'Cameron Street', 'lat': 35.7904985, 'lng': -78.65862159999999, 'labeledLatLngs': [{'label': 'display', 'lat': 35.7904985, 'lng': -78.65862159999999}], 'distance': 162, 'postalCode': '27605', 'cc': 'US', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['432 Woodburn Rd (Cameron Street)', 'Raleigh, NC 27605', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d1c0941735', 'name': 'Mediterranean Restaurant', 'pluralName': 'Mediterranean Restaurants', 'shortName': 'Mediterranean', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}
{'id': '5851e3f5730a9222e2d57826', 'name': 'CHOPT', 'location': {'address': '420 Daniels St', 'lat': 35.78972156368618, 'lng': -78.66073683585822, 'labeledLatLngs': [{'label': 'display', 

...

{'id': '4ad91c4ff964a520311821e3', 'name': 'NOFO @ the Pig', 'location': {'address': '2014 Fairview Rd', 'crossStreet': 'at Five Points', 'lat': 35.80508061463781, 'lng': -78.6468957289808, 'labeledLatLngs': [{'label': 'display', 'lat': 35.80508061463781, 'lng': -78.6468957289808}], 'distance': 724, 'postalCode': '27608', 'cc': 'US', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['2014 Fairview Rd (at Five Points)', 'Raleigh, NC 27608', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d16d941735', 'name': 'Café', 'pluralName': 'Cafés', 'shortName': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}, 'primary': True}], 'delivery': {'id': '1312861', 'url': 'https://www.grubhub.com/restaurant/nofo--the-pig-2014-fairview-road-raleigh/1312861?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=1312861', 'provider': {'name': 'grubhub', 'icon': {'pref

{'id': '4ee404309911944900b07ad5', 'name': 'Bida Manda Laotian Restaurant and Bar', 'location': {'address': '222 S Blount St', 'lat': 35.77722725876699, 'lng': -78.63681194044996, 'labeledLatLngs': [{'label': 'display', 'lat': 35.77722725876699, 'lng': -78.63681194044996}], 'distance': 2632, 'postalCode': '27601', 'cc': 'US', 'neighborhood': 'Downtown Raleigh, Raleigh, NC', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['222 S Blount St', 'Raleigh, NC 27601', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d142941735', 'name': 'Asian Restaurant', 'pluralName': 'Asian Restaurants', 'shortName': 'Asian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/asian_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}, 'venuePage': {'id': '411258596'}}
{'id': '4e091acc1f6e21103396e069', 'name': "Beasley's Chicken + Honey", 'location': {'address': '237 S Wilmington St', 'crossStreet': 'at E Martin St', 'lat': 35.7

{'id': '4ad91c4ff964a520311821e3', 'name': 'NOFO @ the Pig', 'location': {'address': '2014 Fairview Rd', 'crossStreet': 'at Five Points', 'lat': 35.80508061463781, 'lng': -78.6468957289808, 'labeledLatLngs': [{'label': 'display', 'lat': 35.80508061463781, 'lng': -78.6468957289808}], 'distance': 520, 'postalCode': '27608', 'cc': 'US', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['2014 Fairview Rd (at Five Points)', 'Raleigh, NC 27608', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d16d941735', 'name': 'Café', 'pluralName': 'Cafés', 'shortName': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}, 'primary': True}], 'delivery': {'id': '1312861', 'url': 'https://www.grubhub.com/restaurant/nofo--the-pig-2014-fairview-road-raleigh/1312861?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=1312861', 'provider': {'name': 'grubhub', 'icon': {'pref

{'id': '52251fd611d21c8d037f3127', 'name': 'José and Sons', 'location': {'address': '327 - 102 W Davie St', 'lat': 35.77541026824044, 'lng': -78.64478241052932, 'labeledLatLngs': [{'label': 'display', 'lat': 35.77541026824044, 'lng': -78.64478241052932}], 'distance': 69, 'postalCode': '27601', 'cc': 'US', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['327 - 102 W Davie St', 'Raleigh, NC 27601', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}], 'delivery': {'id': '1017900', 'url': 'https://www.grubhub.com/restaurant/jose-and-sons-327-w-davie-st-ste-102-raleigh/1017900?affiliate=1131&utm_source=foursquare-affiliate-network&utm_medium=affiliate&utm_campaign=1131&utm_content=1017900', 'provider': {'name': 'grubhub', 'icon': {'prefix':

{'id': '4b421013f964a52026cc25e3', 'name': 'Jasmin Mediterranean Bistro', 'location': {'address': '2430 Hillsborough St', 'crossStreet': 'at Horne St', 'lat': 35.78784426618954, 'lng': -78.66795586235004, 'labeledLatLngs': [{'label': 'display', 'lat': 35.78784426618954, 'lng': -78.66795586235004}], 'distance': 864, 'postalCode': '27607', 'cc': 'US', 'city': 'Raleigh', 'state': 'NC', 'country': 'United States', 'formattedAddress': ['2430 Hillsborough St (at Horne St)', 'Raleigh, NC 27607', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d1c0941735', 'name': 'Mediterranean Restaurant', 'pluralName': 'Mediterranean Restaurants', 'shortName': 'Mediterranean', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}
{'id': '56bddf3a498ef271f5443985', 'name': 'Jubala Coffee', 'location': {'address': '2100 Hillsborough St', 'lat': 35.7866975976946, 'lng': -78.66326997522265, 'labeledLat